# Tutorial 2: Functions, String and Array Manipulations

In this tutorial, we will be exploring how to manipulate, slice, and work with data stored in arrays, lists, and strings, through the use of functions. We'll be applying these techniques to astronomical specrta (discussed below)

In [ ]:
#RUN THIS CELL BLOCK!!!!!
import numpy as np
import matplotlib.pyplot as plt
import pyfits as pf
%matplotlib inline
!pip install -U okpy 
from client.api.notebook import Notebook
ok = Notebook('hw2.ok')

# Astronomical Spectra: Slicing Practice

Astronomers basically have one way of learning about the universe: **detecting photons**. Pretty much everything we know in astrophysics comes from the analysis of data which is nothing more than how many photons we received from a certain location in the sky. Many of us grew up seeing images, of astrophysical objects like galaxies taken by the Hubble Space Telescope (HST). Those images are useful, and drawing information out of how many photons are in a given pixel bin is what's known as **photometry**. However, there is another really important way of learning about astrophysical objects via their photons. Astronomers take **spectra** of objects to determine everything from their elemental composition to their radial velocity to the amount of dust or obscured star formation going on, etc.

A spectrum is basically what you see when you use a prism to spread out white light into a rainbow. Advanced spectrometers use what is known as an echelle grating to split up light and 'bin' it by wavelength, allowing us to see at every wavelength how many photons we receive from a given source. Almost no astrophysical source in the universe emits a flat (equal strength at all wavelengths) spectrum.

In the cell below, we load a **FITS** file (we'll get to those later) containing a solar spectrum, which ends up displaying the plot below. Let's see how we can slice and dice this image to get out a 1 dimensional spectrum!

In [ ]:
fits_file = pf.open('solar_spec.fits')[0] #use the pyfits library to load the file
image = fits_file.data #extract the 2D CCD Image of the spectra
plt.imshow(image,cmap='gray_r') #Display the raw image

**Question 1a**

What we see above doesn't really look like much of anything, yet. We are actually looking at exactly what our 2-dimensional CCD chip (like the ones in your phone, but fancier) picked up. Our spectograph did something funny, for reasons you'll learn in Astro 120, and split the light into these black horizontal bands we see (Note: the cmap='gray_r' command reversed the image such that dark represents areas of high light flux and white represents low flux. So the light is in the thin black bands, not the white spaces between them). Believe it or not, the spectra are actually hiding in those bands. We need to figure out how to extract just one of those bands to look at (at least at one time). 

To get an idea of how wide the individual bands are (from now on we'll call them "orders"), set a variable called ```vertical_profile``` equal to all the values forming the 500th (center) column (by index) of the image. What do you expect this to look like? try plotting the values you just extracted by typing ```plt.plot(vertical_profile)``` followed by ```plt.show()``` 
under your variable declaration in the box below, and then run it to see.  

In [ ]:
#your code here

In [ ]:
_ = ok.grade('q01a')
_ = ok.backup()

Was this what you expected to see? What we should be seeing here is the brightness of the image as a function of vertical position, for a single column. Everywhere there's a black stripe corresponding to an order of light, we see a spike in the number of photons the detector picks up. We can even note how the spikes get progressively higher towards the right of our plot, just as the bars get thicker and darker towards the bottom of the image we plotted above. 

This was helpful, but we really want to zoom in and figure out the edges of a single order. Let's go with the spike that looks like it's at around 700 on the plot you (hopefully) made. Make a variable called ```smaller_slice``` below and set it to index your ```vertical_profile``` array from around 690 to 710, to catch the peak I just mentioned. Plot it as well, to see that you're catching one full peak, and not too much noise around it (though in a second that noise isn't really gonna matter).

In [ ]:
#your code here


Hopefully, that worked! If you look closely, you'll notice that every time we've sliced into and extracted entries from an array, they get re-indexed from 0 every time. Your plot above should go from 0 to 19 on the x-axis, for example. If we wanted to know what index 10 here corresponded to in the plot of ```vertical_profile``` above, we'd want to take our start index of the slice, 690, and add 10. 

**Question 1b**

We don't have to worry about that problem when using indices from ```vertical_profile``` to index ```image``` because we took the whole column, not a subset, so any indices in ```vertical_profile``` are the same in the image.

That said, it's time to finally extract the solar spectrum! Looking at the plot of ```smaller_slice``` we can see that the row 690 plus ~ 11 of the image will definitely be a solid line of pixels to choose, and will definitely contain the solar data we are after. 

Set a variable ```row_slice``` to the 701st (by index) row of the original image, and plot the result below. If you forgot how to index for multiple rows or columns at once, check out the textbook ;) 

In [ ]:
#your code here

In [ ]:
_ = ok.grade('q01b')
_ = ok.backup()

Congrats! You've just extracted your first spectrum! Well, technically, what we are looking at isn't a spectrum because we are viewing the number of photons (or flux) as a function of pixel number on the CCD, not wavelength, We'd have to go through some conversions to get all the way there, but the basic principles are all here. 

Ultimately, loading a single row of pixels like we've done here can be pretty dangerous, especially when we have more than one row of pixels per order with good data in it (and the more we use, the smaller our uncertainties). In fact, if you look at the plot of your spectrum you'll see it tapers off rather steeply on the right hand side. If you go back to the image of the CCD, you'll see that many of the orders are tilted, and get weaker from one side to the other. By taking a single row, we open ourselves up to those things really affecting our results. So, we've provided one step toward a better option- what if we choose a "box" around the order we are after, and take the vertical average within that box for every column? The result of that idea is set up for you in the box below. Run it, and see if you can figure out the syntax being used to accomplish it. Can you think of any other ideas for improving on our extraction of these spectra?

In [ ]:
flattened_spectrum = image[695:708,:].mean(axis=0)
plt.plot(flattened_spectrum)
plt.show()

**Question 1c**

Now that we have a spectrum to work with, we can start performing some of the corrections and adjustments we would make before displaying it in a paper or performing analysis. 

The first thing we notice about the flattened spectrum above is that there's this weird, precipitous drop near the end. That's not real; the drop in intensity is due internal optics within the system. Essentially, this order receives no light in its last several pixels. Since they aren't data, and are making it hard to see what is happening in the spectrum, let's remove them. 

I've done some tinkering, and it seems like the last 20 pixels should be dropped from the spectrum. Set ```flattened_spectrum``` below to be itself, but indexed to not include the last 20 values (negative indices might be helpful here!). Then replot it to see what it looks like!

In [ ]:
#your code here 

In [ ]:
_ = ok.grade('q01c')
_ = ok.backup()

What we see here are two components of the spectrum: a continuum component, and the spectral lines. The continuum is the broad "bump" shape of the spectrum, the shape it would be if the individual dips weren't there. Those dips are known as **absorption features** or **absorption lines** and are the part of the spectrum that contain information about the chemical elements in the plasma at the surface of the sun, as well as the doppler shifts that may be associated with the gas (whether it is moving towards or away from us). So we want to actually subtract off the continuum emmission (the broad shape) and get a flat spectrum. This is a little tricky, so we'll do this step for you, but try to follow what we are doing in the comments!

In [ ]:
#Start by fitting a third degree polynomial to the spectrum as a rough estimate for the shape of the curve (usually this is sufficient)
x = np.arange(len(flattened_spectrum)) #plotting without x basically just creates this for you
fit = np.polyfit(x,flattened_spectrum,3) #specify third order fit (we will go over fits in 2 weeks)
a = fit[0] #extract the coefficients
b = fit[1] #extract the coefficients
c = fit[2] #extract the coefficients
d = fit[3] #extract the coefficients

y_fit = a*x**3 + b*x**2 + c*x + d #set up a y array with the polynomial equation

plt.plot(x,flattened_spectrum,label='Raw Spectrum')
plt.plot(x,y_fit,'r',lw=2,label='Cubic Polynomial Fit')
plt.legend()
plt.show()

We can see that the polynomial does a decent job fitting the continuum, though it slightly underpredicted the continuum level, so I added in an extra constant offset (the plus 300) to get it closer. In reality we'd have to tinker a bit more, but for our purposes here this is fine.

**Question 1d**

Now that we have a reasonable fit to the data we want to subtract off the predicted continuum at every pixel value from our flattened spectrum. Since everything is in numpy arrays, this is super easy. 

In the code block below, set a variable ```continuum_subtracted_spectrum``` to be the flattened spectrum with the fit continuum value for each pixel (stored in ```y_fit```) subtracted off, and then plot it. 

In [ ]:
#your code here

In [ ]:
_ = ok.grade('q01d')
_ = ok.backup()

We should be able to see from the small "wavy" pattern here that we didn't do a perfect job zeroing out the continuum emission, but that's alright. We now have what any astronomer would instantly recognize as a line spectrum of an astrophysical source. We could now start doing science, like calculating line widths and depths, and figuring out what elements we have in this gas and whether or not it is doppler shifted! 

# Writing Functions

So far in this class, we have been defining variables and either plotting them or printing them to learn something about their values. However, writing your own functions makes your code shorter, easier to debug (more on that in a bit), and easier to follow. In practice, almost any task you are going to perform more than once on different variables deserves a function; it really will make things easier. 

Lets look at an example. Say I had just read in a catalog of exoplanets discovered by the Kepler mission. Upon extracting the "ID/Name" column of the data to try to get the name for each exoplanet, I discover that they list the name in the form of a string that looks like this: '20160207KOI3715b_obs17-18-19'. That is a bit unweildy, even though all the information within that string is probably useful to us in some way or another, such as the date of observation (02/07/2016) and observation numbers (17,18,19). But we are only interested in the name right now, for our current project- so we want to extract the 'KOI3715b' part of the string and nothing else. 

Well, looking at the string we extracted, we might be tempted to just write something like ```name_extracted=nameidstr[8:16]```, remembering that when indexing, it includes the start index and stops one before the end index. But this is dangerous for several reasons. First, I have like, five-hundred or a thousand or maybe even more names to extract. Doing them all one by one wouldn't make much sense. Moreover, there's no inherent guarantee that the strings are all alike, that is, that the index the name starts on and ends on is the same in every string. There must be **some** convention- the data is **designed** to be parsed. So I'm gonna write a function to do it. 

In [2]:
def find(string, char): # Don't worry about while and ifs here, we will get to those. This simply finds a character
    index = 0           # in a string and returns its index within the string 
    while index < len(string):
        if string[index] == char:
            return index
        index += 1
    return -1

def extract_name(string):
    start_index = find(string,'K')
    end_index = find(string,'_')
    extracted = string[start_index:end_index]
    return extracted


name1 = '20160207KOI3715b_obs17-18-19'
short_name1 = extract_name(name1)
print(short_name1)

KOI3715b


The above is a great example of how we can not only write functions to accomplish tasks, but we can call functions from other functions (notice how I used my find function within my extract name function) and even have functions inside functions if we want. All of this helps modularlize our code into more managable pieces, and makes our code more general- a well written function can be applied to all sorts of information- it's just a pipeline for turning inputs to outputs. Let's get a little bit of practice writing some functions.

**Question 2**

Write a function called string_explosion that takes in a non-empty string like "Code" and returns a long string containing every prefix of the input. That is, building the word using the first character, then the first two characters, then the first three, etc. For example:

in: string_splosion('Code')

out: 'CCoCodCode'

in: string_splosion('data!')

out: 'ddadatdatadata!'

in: string_splosion('hi')

out: 'hhi'

(adapted from DS100)

In [ ]:
#your code here

In [ ]:
_ = ok.grade('q02')
_ = ok.backup()

**Question 3**

Finish the function below which solves the quadratic equation (the dreadful thing we all learned in high school, now barely use, but groan really loudly if we have to). In case you need a reminder, the formula is 
$$x =\frac{ -b \pm \sqrt{b^2 + 4ac}}{2a} $$

We've set up the structure of the function for you, with some documentation (in the quotes) which, if someone ran a help function on your function would pull up that documentation. In that documentation you'll notice it specifies what happens if the discriminant is negative. numpy's sqrt function doesn't like negatives, and won't just spit out a nice 'i' for you. If someone were running many calculations, they'd want to know when something didn't have real roots, so we've put in an error message for that case. Your code should be where the ellipses are, for the case when the descriminant is >=0. 

In [ ]:
def quad_solve(a,b,c):
    '''
    A function to solve the quadratic formula for any a,b,c.
    Parameters
    --------------
    a (float): quadratic coefficient 
    b (float): linear coefficient
    c (float): constant coefficient
    
    Returns
    -------------
    x1 (positive case), x2 (negative case) (float): The solution to the quadratic fomula, both values of x should both exist, or 1 if only one exists
    OR
    x (float): if only one intercept exists
    Exceptions
    -------------
    If the discriminant is < 0 (no real roots), None is returned. 
    ''' 
    discriminant = b**2 + 4*a*c
    if discriminant < 0:
        # Discriminant was not positive, returning None 
        return None
    elif discriminant==0:
        ...
        return x
    else:
        ...
        ...
        return x1, x2
    

In [ ]:
_ = ok.grade('q03')
_ = ok.backup()

Great! You are done with the tutorial! Follow the cells below to submit your assignment. 

In [ ]:
_ = ok.grade_all() #Grade all the tests to make sure you haven't missed anything.

In [ ]:
# Now, we'll submit to okpy
_ = ok.submit()